In [1]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.decoder import Decoder, DecoderAttention, DecoderTransformerBlock


In [2]:
img_dims = [256, 256]
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, batch_size=1, device=torch.device('mps'))
batch = next(iter(data.train))
vocab_size = len(data.tokenizer.get_vocab())
enc_output = torch.randn(1, 512, 512)
model = Decoder(512, 256, vocab_size, .75)
model(batch[1]['input_ids'], enc_output, mask=batch[1]['attention_mask']).shape

torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])


torch.Size([1, 256, 2160])

In [3]:
batch[1]['attention_mask'].dtype

torch.int64

In [3]:
batch[1]

{'input_ids': tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,

In [3]:
HELP = DecoderTransformerBlock(512, 8, .70)
inpu = torch.randn(1, 16, 512)
HELP(inpu, inpu, inpu).shape

torch.Size([1, 16, 512])

In [15]:
batch = next(iter(data.train))
tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json')
tokens = list(map(lambda x: x.data, batch[1]['input_ids'][0]))
dec = [tokenizer.decode(tok) for tok in tokens]
print(tokens)
print(''.join([detok.replace('Ġ', ' ') for detok in dec]))
#tokens

[tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)

In [28]:
class TransformerDecoder(nn.Module):
    def __init__(self, n_embd, dropout=.75, block_size=256, num_blocks=6, num_heads=8):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.zeros(1, block_size, n_embd))
        self.transformer_blocks = nn.ModuleList(
            [
                TransformerBlock(n_embd, num_heads, dropout) for _ in range(num_blocks)
            ]
        )
        

In [3]:
x = torch.randn(1, 256, 512)
x.mean(dim=1).shape

torch.Size([1, 512])